In [1]:
import csv
from uncertainties import ufloat

# Cargar datos del archivo CSV
data = []
with open('../../Downloads/phot_mag_err_EBV.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Convertir las columnas numéricas a float
        row['mag_g'] = float(row['mag_g']); row['sigma_g'] = float(row['sigma_g'])
        row['mag_i'] = float(row['mag_i']); row['sigma_i'] = float(row['sigma_i'])
        row['E(B-V)'] = float(row['E(B-V)']); row['E(B-V)_ERR'] = float(row['E(B-V)_ERR'])
        data.append(row)

# Ejemplo: crear ufloat para el primer objeto (ID=1) y mostrar sus valores
obj1 = data[0]
mg1 = ufloat(obj1['mag_g'], obj1['sigma_g'])   # magnitud g con error
mi1 = ufloat(obj1['mag_i'], obj1['sigma_i'])   # magnitud i con error
EBV1 = ufloat(obj1['E(B-V)'], obj1['E(B-V)_ERR'])  # E(B-V) con error
print(f"Objeto ID1 - m_g = {mg1}, m_i = {mi1}, E(B-V) = {EBV1}")


Objeto ID1 - m_g = 16.6370+/-0.0004, m_i = 16.9986+/-0.0009, E(B-V) = 0.109+/-0.013


In [2]:
import uncertainties.umath as umath  # funciones matemáticas compatibles con ufloat

# Constante R_V de Calzetti
Rv = 4.05

# Longitudes de onda en micrones
lambda_g = 4770.8 / 1e4   # 4770.8 Å = 0.47708 μm
lambda_i = 7774.2 / 1e4   # 7774.2 Å = 0.77742 μm

# Calcular k(lambda) para cada filtro usando la ley de Calzetti
if lambda_g < 0.63:
    k_g = 2.659 * (-2.156 + 1.509/lambda_g - 0.198/(lambda_g**2) + 0.011/(lambda_g**3)) + Rv
if lambda_i >= 0.63:
    k_i = 2.659 * (-1.857 + 1.040/lambda_i) + Rv

print(f"k'(λ_g = 4770.8 Å) = {k_g:.3f}")
print(f"k'(λ_i = 7774.2 Å) = {k_i:.3f}")


k'(λ_g = 4770.8 Å) = 4.684
k'(λ_i = 7774.2 Å) = 2.669


In [4]:
# Recorrer cada objeto y calcular A_g, A_i, m_g0, m_i0 con incertidumbres
results = []
for obj in data:
    mg = ufloat(obj['mag_g'], obj['sigma_g'])
    mi = ufloat(obj['mag_i'], obj['sigma_i'])
    EBV = ufloat(obj['E(B-V)'], obj['E(B-V)_ERR'])
    # Extinción en cada filtro
    A_g = k_g * EBV
    A_i = k_i * EBV
    # Magnitudes corregidas por extinción
    m_g0 = mg - A_g
    m_i0 = mi - A_i
    # Guardar resultados (podemos almacenar ufloat o sus nominales y desviaciones)
    results.append({
        'ID': obj['ID'],
        'A_g': A_g, 'A_i': A_i,
        'm_g0': m_g0, 'm_i0': m_i0
    })
    # Imprimir resultados para este objeto
    print(f"ID {obj['ID']}: A_g = {A_g:.3f}, A_i = {A_i:.3f}, "
          f"m_g0 = {m_g0:.3f}, m_i0 = {m_i0:.3f}")


ID 1: A_g = 0.511+/-0.061, A_i = 0.291+/-0.035, m_g0 = 16.126+/-0.061, m_i0 = 16.708+/-0.035
ID 2: A_g = 0.253+/-0.075, A_i = 0.144+/-0.043, m_g0 = 17.092+/-0.075, m_i0 = 17.111+/-0.043
ID 3: A_g = 0.501+/-0.080, A_i = 0.286+/-0.045, m_g0 = 16.769+/-0.080, m_i0 = 16.774+/-0.045
ID 4: A_g = 0.206+/-0.061, A_i = 0.117+/-0.035, m_g0 = 16.864+/-0.061, m_i0 = 17.227+/-0.035
ID 5: A_g = 0.590+/-0.075, A_i = 0.336+/-0.043, m_g0 = 16.320+/-0.075, m_i0 = 16.786+/-0.043
ID 6: A_g = 0.131+/-0.070, A_i = 0.075+/-0.040, m_g0 = 16.824+/-0.070, m_i0 = 17.156+/-0.040
ID 7: A_g = 0.253+/-0.061, A_i = 0.144+/-0.035, m_g0 = 18.344+/-0.061, m_i0 = 18.574+/-0.035
ID 8: A_g = 0.276+/-0.089, A_i = 0.157+/-0.051, m_g0 = 17.738+/-0.089, m_i0 = 17.873+/-0.051
ID 9: A_g = 0.829+/-0.094, A_i = 0.472+/-0.053, m_g0 = 16.516+/-0.094, m_i0 = 16.628+/-0.053
ID 10: A_g = 0.155+/-0.066, A_i = 0.088+/-0.037, m_g0 = 17.369+/-0.066, m_i0 = 17.135+/-0.037
ID 11: A_g = 0.187+/-0.075, A_i = 0.107+/-0.043, m_g0 = 18.154+/-0.07

In [5]:
import math
from uncertainties import ufloat

# --- Módulo de distancia (distancia fija: 4.19 Mpc) ---
distance_pc = 4.19e6  # 4.19 Mpc en parsec
print(distance_pc)

# --- Magnitud absoluta en i: M_i = m_i0 - 5*log10(distance_pc) + 5 ---
for res in results:
    M_i = res['m_i0'] - 5.0 * math.log10(distance_pc) + 5.0  # ufloat - float + float
    res['M_i'] = M_i

    # Imprimir con mayor precisión
    print(f"ID {res['ID']}: M_i = {M_i.nominal_value:.6f} ± {M_i.std_dev:.6f}")


4190000.0
ID 1: M_i = -11.403388 ± 0.034713
ID 2: M_i = -10.999574 ± 0.042733
ID 3: M_i = -11.336639 ± 0.045389
ID 4: M_i = -10.884471 ± 0.034725
ID 5: M_i = -11.325246 ± 0.042723
ID 6: M_i = -10.955042 ± 0.040052
ID 7: M_i = -9.537214 ± 0.034865
ID 8: M_i = -10.237741 ± 0.050756
ID 9: M_i = -11.482663 ± 0.053398
ID 10: M_i = -10.976198 ± 0.037389
ID 11: M_i = -9.683314 ± 0.042967
ID 12: M_i = -9.186882 ± 0.040735
ID 13: M_i = -9.169694 ± 0.061770
ID 14: M_i = -8.371232 ± 0.059605
ID 15: M_i = -9.576522 ± 0.037533
ID 16: M_i = -8.939662 ± 0.061689
ID 17: M_i = -9.491579 ± 0.074807
ID 19: M_i = -8.302145 ± 0.160404
ID 20: M_i = -8.139514 ± 0.269829


In [6]:
# --- Masa estelar estimada con la ecuación (8) de Taylor+2011 ---
for res in results:
    # Color g-i con corrección por extinción
    color_gi = res['m_g0'] - res['m_i0']
    # log10(M*/M☉) según Taylor+2011 (ecuación 8)
    logM = 1.15 + 0.70 * color_gi - 0.4 * res['M_i']
    res['logM'] = logM
    # Masa en unidades de M☉ (propagando la incertidumbre)
    M_star = 10**logM
    res['M_star'] = M_star
    # Imprimir resultado con su incertidumbre
    print(f"ID {res['ID']}: log10(M/M\u2609) = {logM.nominal_value:.6f} ± {logM.std_dev:.6f}")


ID 1: log10(M/M☉) = 5.304481 ± 0.004571
ID 2: log10(M/M☉) = 5.536318 ± 0.005717
ID 3: log10(M/M☉) = 5.680673 ± 0.005931
ID 4: log10(M/M☉) = 5.249753 ± 0.004685
ID 5: log10(M/M☉) = 5.354287 ± 0.005617
ID 6: log10(M/M☉) = 5.299662 ± 0.005257
ID 7: log10(M/M☉) = 4.804010 ± 0.005855
ID 8: log10(M/M☉) = 5.150551 ± 0.006887
ID 9: log10(M/M☉) = 5.664698 ± 0.006963
ID 10: log10(M/M☉) = 5.704092 ± 0.004987
ID 11: log10(M/M☉) = 4.831911 ± 0.007580
ID 12: log10(M/M☉) = 4.631892 ± 0.009787
ID 13: log10(M/M☉) = 4.734934 ± 0.010927
ID 14: log10(M/M☉) = 4.788941 ± 0.013892
ID 15: log10(M/M☉) = 4.662995 ± 0.006198
ID 16: log10(M/M☉) = 4.497441 ± 0.010390
ID 17: log10(M/M☉) = 5.057707 ± 0.010250
ID 19: log10(M/M☉) = 4.531383 ± 0.022863
ID 20: log10(M/M☉) = 4.243945 ± 0.036738


In [7]:
import csv

# Suponiendo que ya existen las listas 'data' (filas originales) y 'results' (resultados calculados) de celdas previas

# Combinar columnas originales con las nuevas columnas calculadas
filas_salida = []
for obj_orig, obj_calc in zip(data, results):
    fila = {
        # columnas originales
        'ID': obj_orig['ID'],
        'mag_g': obj_orig['mag_g'],
        'sigma_g': obj_orig['sigma_g'],
        'mag_i': obj_orig['mag_i'],
        'sigma_i': obj_orig['sigma_i'],
        'E(B-V)': obj_orig['E(B-V)'],
        'E(B-V)_ERR': obj_orig['E(B-V)_ERR'],
        # nuevas columnas calculadas (valor nominal e incertidumbre)
        'A_g': obj_calc['A_g'].nominal_value,        # extinción banda g
        'sigma_A_g': obj_calc['A_g'].std_dev,        # incertidumbre de A_g
        'A_i': obj_calc['A_i'].nominal_value,        # extinción banda i
        'sigma_A_i': obj_calc['A_i'].std_dev,        # incertidumbre de A_i
        'm_g0': obj_calc['m_g0'].nominal_value,      # magnitud g corregida
        'sigma_m_g0': obj_calc['m_g0'].std_dev,      # incertidumbre de m_g0
        'm_i0': obj_calc['m_i0'].nominal_value,      # magnitud i corregida
        'sigma_m_i0': obj_calc['m_i0'].std_dev,      # incertidumbre de m_i0
        'M_i': obj_calc['M_i'].nominal_value,        # magnitud absoluta i
        'sigma_M_i': obj_calc['M_i'].std_dev,        # incertidumbre de M_i
        'logM': obj_calc['logM'].nominal_value,      # log10(M/M☉)
        'sigma_logM': obj_calc['logM'].std_dev,      # incertidumbre de logM
        'M_star': obj_calc['M_star'].nominal_value,  # masa estelar en M☉
        'sigma_M_star': obj_calc['M_star'].std_dev   # incertidumbre de M_star
    }
    filas_salida.append(fila)

# Especificar las columnas en el orden deseado (opcional, para asegurar el orden en el CSV)
columnas = list(filas_salida[0].keys())

# Escribir a un nuevo archivo CSV en la ruta deseada (misma carpeta que el original)
ruta_salida = '../../Downloads/mag_err_phot_resultados.csv'  # puedes cambiar nombre si lo deseas
with open(ruta_salida, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=columnas)
    writer.writeheader()
    for fila in filas_salida:
        writer.writerow(fila)

print(f"Archivo CSV guardado en {ruta_salida} con {len(columnas)} columnas (originales + nuevas).")


Archivo CSV guardado en ../../Downloads/mag_err_phot_resultados.csv con 21 columnas (originales + nuevas).
